In [7]:
import pandas as pd
import numpy as np

# 載入訓練資料與標籤
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')


print(X_train.isnull().sum())

Id             0
鄉鎮市區           0
交易標的           0
路名             0
土地移轉總面積平方公尺    0
都市土地使用分區       0
土地數            0
建物數            0
車位數            0
移轉層次           0
移轉層次項目         0
總樓層數           0
建物型態           0
主要用途           0
主要建材           0
建築完成年月         0
建物移轉總面積平方公尺    0
建物現況格局-房       0
建物現況格局-廳       0
建物現況格局-衛       0
建物現況格局-隔間      0
有無管理組織         0
交易年            0
交易日            0
交易月            0
地鐵站            0
超商             0
公園             0
托兒所            0
國小             0
國中             0
高中職            0
大學             0
金融機構           0
醫院             0
大賣場            0
超市             0
百貨公司           0
警察局            0
消防局            0
縱坐標            0
橫坐標            0
dtype: int64


In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# 處理類別型變數的 One-Hot Encoding
categorical_features = [
    "鄉鎮市區",
    "交易標的",
    "都市土地使用分區",
    "建物型態",
    "主要用途",
    "主要建材",
    "有無管理組織",
]
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)

# 使用平均值填補數值變數的空值
imputer = SimpleImputer(strategy="mean")
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

KeyError: "None of [Index(['鄉鎮市區', '交易標的', '都市土地使用分區', '建物型態', '主要用途', '主要建材', '有無管理組織'], dtype='object')] are in the [columns]"

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 分割訓練集和驗證集
X_train_part, X_val, y_train_part, y_val = train_test_split(X_train, y_train['單價元平方公尺'], test_size=0.2, random_state=42)

# 訓練隨機森林模型
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_part, y_train_part)
rf_val_pred = rf_model.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_val_pred))
print("Random Forest RMSE:", rf_rmse)

# 訓練梯度提升模型
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train_part, y_train_part)
gb_val_pred = gb_model.predict(X_val)
gb_rmse = np.sqrt(mean_squared_error(y_val, gb_val_pred))
print("Gradient Boosting RMSE:", gb_rmse)

In [ ]:
from sklearn.model_selection import GridSearchCV

# 定義參數網格
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

# 使用隨機森林進行網格搜索
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train['單價元平方公尺'])
print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))

In [ ]:
# 載入測試數據並進行相同的處理
X_test = pd.read_csv('X_test.csv')
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
X_test = scaler.transform(X_test)

# 使用最佳模型預測測試集
test_predictions = grid_search.best_estimator_.predict(X_test)

# 生成提交文件
submission = pd.DataFrame({'Id': X_test['Id'], '單價元平方公尺': test_predictions})
submission.to_csv('submission.csv', index=False)